<a href="https://colab.research.google.com/github/MajesticFT/SuperAI2/blob/main/Week1_FontRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library

In [ ]:
import tensorflow as tf
import keras, os
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Activation, BatchNormalization,Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from tqdm import tqdm
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q '/content/drive/Shareddrives/FontRecog/resized.zip'
!unzip -q '/content/drive/Shareddrives/FontRecog/crop_t.zip'

warning [/content/drive/Shareddrives/FontRecog/resized.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [/content/drive/Shareddrives/FontRecog/resized.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #129720).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


In [ ]:
id_test = pd.read_csv('/content/drive/Shareddrives/FontRecog/id_test.csv')
id_test['id']=id_test['id'].apply(lambda x: x+'.png')
print(id_test.head(5))

             id
0  b68ab614.png
1  2d3a3417.png
2  b9b73e2d.png
3  1d8598b0.png
4  6626584d.png


In [ ]:

stats = pd.read_csv('/content/drive/Shareddrives/FontRecog/stats.csv')
stats['id']=stats['id'].apply(lambda x: x+'.png')
stats_font=stats[['id', 'font']]
print(stats_font.head(5))

             id                     font
0  100045cf.png     FC_Knomphing_Regular
1  10006750.png  DM_Shining_Star_Regular
2  1000bc2d.png               fonttintin
3  10019de9.png               TH_Sarabun
4  1001fcf9.png               fonttintin


In [ ]:
print(stats_font['font'].unique())

['FC_Knomphing_Regular' 'DM_Shining_Star_Regular' 'fonttintin'
 'TH_Sarabun' 'Cordia_New' 'Angsana_New' 'TH_Chakra_Petch' 'Kunlasatri'
 'TH_Charm_of_AU' 'TH_Mali_Grade6']


# Font

In [ ]:
datagen=ImageDataGenerator(rescale=1./255., validation_split=0.25)
test_datagen=ImageDataGenerator(rescale=1./255.)
directory = '/content/resized'

Import Data into model using ImageDataGenerator

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=stats_font,
directory=directory,
x_col="id",
y_col='font',
batch_size=256,
seed=42,
subset='training',
color_mode='grayscale',
class_mode="sparse",
shuffle=True,
target_size=(112,112))

valid_generator=datagen.flow_from_dataframe(
dataframe=stats_font,
directory=directory,
x_col='id',
y_col='font',
color_mode='grayscale',
subset='validation',
batch_size=256,
seed=42,
shuffle=True,
class_mode="sparse",
target_size=(112,112))

test_generator=test_datagen.flow_from_dataframe(
dataframe=id_test,
directory='/content/crop_t',
x_col="id",
y_col=None,
color_mode='grayscale',
batch_size=256,
seed=42,
shuffle=False,
class_mode=None,
target_size=(112,112))

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 8161 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 91170 validated image filenames belonging to 10 classes.
Found 30390 validated image filenames belonging to 10 classes.
Found 95997 validated image filenames.


Model (Modified from VGG16)

In [ ]:

model = Sequential()
model.add(Conv2D(input_shape=(112,112,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=512,activation="relu"))
model.add(Dense(units=512,activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(units=10, activation="softmax"))

In [ ]:

model.compile(optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 112, 112, 64)      640       
                                                                 
 conv2d_7 (Conv2D)           (None, 112, 112, 32)      18464     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 28, 64)       

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

Epoch 1/10
178/178 [==============================] - 221s 1s/step - loss: 1.9147 - accuracy: 0.3019 - val_loss: 1.3403 - val_accuracy: 0.5289
Epoch 2/10
178/178 [==============================] - 217s 1s/step - loss: 1.0378 - accuracy: 0.6325 - val_loss: 0.7950 - val_accuracy: 0.7200
Epoch 3/10
178/178 [==============================] - 217s 1s/step - loss: 0.6421 - accuracy: 0.7716 - val_loss: 0.5849 - val_accuracy: 0.7919
Epoch 4/10
178/178 [==============================] - 218s 1s/step - loss: 0.4661 - accuracy: 0.8359 - val_loss: 0.4363 - val_accuracy: 0.8470
Epoch 5/10
178/178 [==============================] - 219s 1s/step - loss: 0.3417 - accuracy: 0.8770 - val_loss: 0.4116 - val_accuracy: 0.8567
Epoch 6/10
178/178 [==============================] - 218s 1s/step - loss: 0.2747 - accuracy: 0.9004 - val_loss: 0.3887 - val_accuracy: 0.8689
Epoch 7/10
178/178 [==============================] - 218s 1s/step - loss: 0.2196 - accuracy: 0.9200 - val_loss: 0.3512 - val_accuracy: 0.8808

Evaluation

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_TEST)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


[0.3668203353881836, 0.8906219005584717]

In [ ]:
test_generator.reset()
pred=model.predict(test_generator,
steps=STEP_SIZE_TEST+1,
verbose=1)

188/188 [==============================] - 68s 362ms/step


In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames=test_generator.filenames
print(len(filenames))
print(len(predictions))
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("/content/results.csv",index=False)
model.save('/content/models/model_font.h5')

95997
95997


# Size

In [ ]:
stats_size = stats[['id', 'size']]
print(stats_size.head(5))

             id  size
0  100045cf.png  12px
1  10006750.png  16px
2  1000bc2d.png  16px
3  10019de9.png  14px
4  1001fcf9.png  14px


In [ ]:
print(stats_size['size'].nunique())

7


In [ ]:
train_generator_size=datagen.flow_from_dataframe(
dataframe=stats_size,
directory=directory,
x_col="id",
y_col='size',
batch_size=256,
seed=42,
subset='training',
color_mode='grayscale',
class_mode="sparse",
shuffle=True,
target_size=(112,112))

valid_generator_size=datagen.flow_from_dataframe(
dataframe=stats_size,
directory=directory,
x_col='id',
y_col='size',
subset='validation',
color_mode='grayscale',
batch_size=256,
seed=42,
shuffle=True,
class_mode="sparse",
target_size=(112,112))

test_generator_size=test_datagen.flow_from_dataframe(
dataframe=id_test,
directory='/content/crop_t',
color_mode='grayscale',
x_col="id",
y_col=None,
batch_size=256,
seed=42,
shuffle=False,
class_mode=None,
target_size=(112,112))

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 8161 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 91170 validated image filenames belonging to 7 classes.
Found 30390 validated image filenames belonging to 7 classes.
Found 95997 validated image filenames.


In [ ]:
model_size = Sequential()

model_size.add(Conv2D(input_shape=(112,112,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_size.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model_size.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_size.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model_size.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_size.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model_size.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model_size.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_size.add(Dropout(0.25))
model_size.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_size.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_size.add(Flatten())
model_size.add(Dense(units=512,activation="relu"))
model_size.add(Dense(units=512,activation="relu"))
model_size.add(Dropout(0.25))
model_size.add(Dense(units=7, activation="softmax"))

In [ ]:
model_size.compile(optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model_size.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 112, 112, 64)      640       
                                                                 
 conv2d_13 (Conv2D)          (None, 112, 112, 32)      18464     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 28, 28, 64)       

In [ ]:
STEP_SIZE_TRAIN_SIZE=train_generator_size.n//train_generator_size.batch_size
STEP_SIZE_VALID_SIZE=valid_generator_size.n//valid_generator_size.batch_size
STEP_SIZE_TEST_SIZE=test_generator_size.n//test_generator_size.batch_size
model_size.fit(train_generator_size,
                    steps_per_epoch=STEP_SIZE_TRAIN_SIZE,
                    validation_data=valid_generator_size,
                    validation_steps=STEP_SIZE_VALID_SIZE,
                    epochs=7)

Epoch 1/7
178/178 [==============================] - 219s 1s/step - loss: 1.7437 - accuracy: 0.3341 - val_loss: 1.5923 - val_accuracy: 0.3863
Epoch 2/7
178/178 [==============================] - 218s 1s/step - loss: 1.3477 - accuracy: 0.4715 - val_loss: 1.1100 - val_accuracy: 0.5714
Epoch 3/7
178/178 [==============================] - 218s 1s/step - loss: 1.0042 - accuracy: 0.6045 - val_loss: 0.8839 - val_accuracy: 0.6576
Epoch 4/7
178/178 [==============================] - 218s 1s/step - loss: 0.8047 - accuracy: 0.6819 - val_loss: 0.7706 - val_accuracy: 0.6953
Epoch 5/7
178/178 [==============================] - 218s 1s/step - loss: 0.6647 - accuracy: 0.7392 - val_loss: 0.6375 - val_accuracy: 0.7497
Epoch 6/7
178/178 [==============================] - 218s 1s/step - loss: 0.5575 - accuracy: 0.7821 - val_loss: 0.5834 - val_accuracy: 0.7739
Epoch 7/7
178/178 [==============================] - 218s 1s/step - loss: 0.4807 - accuracy: 0.8129 - val_loss: 0.5961 - val_accuracy: 0.7770


In [ ]:
model_size.evaluate(valid_generator_size,
steps=STEP_SIZE_TEST_SIZE)

187/187 [==============================] - 21s 107ms/step - loss: 0.5962 - accuracy: 0.7770


[0.5961966514587402, 0.7770318984985352]

In [ ]:
test_generator_size.reset()
pred_size=model_size.predict_generator(test_generator_size,
steps=STEP_SIZE_TEST_SIZE+1,
verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  after removing the cwd from sys.path.


188/188 [==============================] - 62s 332ms/step


In [ ]:
predicted_class_indices_size=np.argmax(pred_size,axis=1)
labels_size = (train_generator_size.class_indices)
labels_size = dict((v,k) for k,v in labels_size.items())
predictions_size = [labels_size[k] for k in predicted_class_indices_size]

In [ ]:
filenames_size=test_generator_size.filenames
results_size=pd.DataFrame({"id":filenames_size,
                      "predicted_size":predictions_size})
results_size.to_csv("/content/results_size.csv",index=False)
model_size.save('/content/models/model_size.h5')

# Style

In [ ]:
stats_style = stats[['id', 'style']]
print(stats_style.head(5))

             id   style
0  100045cf.png  normal
1  10006750.png  italic
2  1000bc2d.png  normal
3  10019de9.png  normal
4  1001fcf9.png  normal


In [ ]:
print(stats_style['style'].nunique())

2


In [ ]:
train_generator_style=datagen.flow_from_dataframe(
dataframe=stats_style,
directory=directory,
x_col="id",
y_col='style',
batch_size=256,
seed=42,
subset='training',
color_mode='grayscale',
class_mode="binary",
shuffle=True,
target_size=(112,112))

valid_generator_style=datagen.flow_from_dataframe(
dataframe=stats_style,
directory=directory,
x_col='id',
y_col='style',
subset='validation',
color_mode='grayscale',
batch_size=256,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(112,112))

test_generator_style=test_datagen.flow_from_dataframe(
dataframe=id_test,
directory='/content/crop_t',
color_mode='grayscale',
x_col="id",
y_col=None,
batch_size=256,
seed=42,
shuffle=False,
class_mode=None,
target_size=(112,112))

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 8161 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 91170 validated image filenames belonging to 2 classes.
Found 30390 validated image filenames belonging to 2 classes.
Found 95997 validated image filenames.


In [ ]:
model_style = Sequential()

model_style.add(Conv2D(input_shape=(112,112,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_style.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model_style.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_style.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model_style.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_style.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model_style.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model_style.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_style.add(Dropout(0.25))
model_style.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_style.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_style.add(Flatten())
model_style.add(Dense(units=512,activation="relu"))
model_style.add(Dense(units=512,activation="relu"))
model_style.add(Dropout(0.25))
model_style.add(Dense(units=1, activation="sigmoid"))

In [ ]:
model_style.compile(optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_style.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 112, 112, 64)      640       
                                                                 
 conv2d_19 (Conv2D)          (None, 112, 112, 32)      18464     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 56, 56, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 28, 28, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 28, 28, 64)       

In [ ]:
STEP_SIZE_TRAIN_STYLE=train_generator_style.n//train_generator_style.batch_size
STEP_SIZE_VALID_STYLE=valid_generator_style.n//valid_generator_style.batch_size
STEP_SIZE_TEST_STYLE=test_generator_style.n//test_generator_style.batch_size
model_style.fit(train_generator_style,
                    steps_per_epoch=STEP_SIZE_TRAIN_STYLE,
                    validation_data=valid_generator_style,
                    validation_steps=STEP_SIZE_VALID_STYLE,
                    epochs=10)

Epoch 1/10
178/178 [==============================] - 219s 1s/step - loss: 0.5234 - accuracy: 0.7061 - val_loss: 0.3545 - val_accuracy: 0.8375
Epoch 2/10
178/178 [==============================] - 219s 1s/step - loss: 0.2620 - accuracy: 0.8834 - val_loss: 0.1870 - val_accuracy: 0.9203
Epoch 3/10
178/178 [==============================] - 218s 1s/step - loss: 0.1721 - accuracy: 0.9264 - val_loss: 0.1349 - val_accuracy: 0.9424
Epoch 4/10
178/178 [==============================] - 219s 1s/step - loss: 0.1189 - accuracy: 0.9496 - val_loss: 0.1043 - val_accuracy: 0.9565
Epoch 5/10
178/178 [==============================] - 219s 1s/step - loss: 0.0890 - accuracy: 0.9634 - val_loss: 0.0933 - val_accuracy: 0.9621
Epoch 6/10
178/178 [==============================] - 219s 1s/step - loss: 0.0678 - accuracy: 0.9725 - val_loss: 0.0786 - val_accuracy: 0.9692
Epoch 7/10
178/178 [==============================] - 219s 1s/step - loss: 0.0659 - accuracy: 0.9741 - val_loss: 0.0793 - val_accuracy: 0.9689

In [ ]:
model_style.evaluate(valid_generator_style,
steps=STEP_SIZE_TEST_STYLE)

187/187 [==============================] - 20s 107ms/step - loss: 0.0734 - accuracy: 0.9748


[0.07340100407600403, 0.974761426448822]

In [ ]:
test_generator_style.reset()
pred_style=model_style.predict(test_generator_style,
steps=STEP_SIZE_TEST_STYLE+1,
verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  after removing the cwd from sys.path.


188/188 [==============================] - 68s 361ms/step


In [ ]:
predicted_class_indices_style=np.argmax(pred_style,axis=1)
labels_style = (train_generator_style.class_indices)
labels_style = dict((v,k) for k,v in labels_style.items())
predictions_style = [labels_style[k] for k in predicted_class_indices_style]

In [ ]:
filenames_style=test_generator_style.filenames
results_style=pd.DataFrame({"id":filenames_style,
                      "predicted_style":predictions_style})
results_style.to_csv("/content/results_style.csv",index=False)
model_style.save('/content/models/model_style.h5')

# Weight

In [ ]:
stats_weight= stats[['id', 'weight']]
print(stats_weight.head(5))

             id  weight
0  100045cf.png  normal
1  10006750.png    bold
2  1000bc2d.png    bold
3  10019de9.png    bold
4  1001fcf9.png    bold


In [ ]:
print(stats_weight['weight'].nunique())

2


In [ ]:
train_generator_weight=datagen.flow_from_dataframe(
dataframe=stats_weight,
directory=directory,
x_col="id",
y_col='weight',
color_mode='grayscale',
batch_size=256,
seed=42,
subset='training',
class_mode="binary",
shuffle=True,
target_size=(112,112))

valid_generator_weight=datagen.flow_from_dataframe(
dataframe=stats_weight,
directory=directory,
x_col='id',
y_col='weight',
subset='validation',
color_mode='grayscale',
batch_size=256,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(112,112))

test_generator_weight=test_datagen.flow_from_dataframe(
dataframe=id_test,
directory='/content/crop_t',
color_mode='grayscale',
x_col="id",
y_col=None,
batch_size=256,
seed=42,
shuffle=False,
class_mode=None,
target_size=(112,112))

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 8161 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 91170 validated image filenames belonging to 2 classes.
Found 30390 validated image filenames belonging to 2 classes.
Found 95997 validated image filenames.


In [ ]:
model_weight = Sequential()

model_weight.add(Conv2D(input_shape=(112,112,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_weight.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model_weight.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_weight.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model_weight.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_weight.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model_weight.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model_weight.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_weight.add(Dropout(0.25))
model_weight.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_weight.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model_weight.add(Flatten())
model_weight.add(Dense(units=512,activation="relu"))
model_weight.add(Dense(units=512,activation="relu"))
model_weight.add(Dropout(0.25))
model_weight.add(Dense(units=1, activation="sigmoid"))

In [ ]:
model_weight.compile(optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_weight.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 112, 112, 64)      640       
                                                                 
 conv2d_25 (Conv2D)          (None, 112, 112, 32)      18464     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 56, 56, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 28, 28, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 28, 28, 64)       

In [ ]:
STEP_SIZE_TRAIN_WEIGHT=train_generator_weight.n//train_generator_weight.batch_size
STEP_SIZE_VALID_WEIGHT=valid_generator_weight.n//valid_generator_weight.batch_size
STEP_SIZE_TEST_WEIGHT=test_generator_weight.n//test_generator_weight.batch_size
model_weight.fit(train_generator_weight,
                    steps_per_epoch=STEP_SIZE_TRAIN_WEIGHT,
                    validation_data=valid_generator_weight,
                    validation_steps=STEP_SIZE_VALID_WEIGHT,
                    epochs=10)

Epoch 1/8
178/178 [==============================] - 220s 1s/step - loss: 0.5921 - accuracy: 0.6622 - val_loss: 0.4766 - val_accuracy: 0.7747
Epoch 2/8
178/178 [==============================] - 218s 1s/step - loss: 0.3991 - accuracy: 0.8166 - val_loss: 0.3294 - val_accuracy: 0.8573
Epoch 3/8
178/178 [==============================] - 218s 1s/step - loss: 0.3200 - accuracy: 0.8593 - val_loss: 0.2920 - val_accuracy: 0.8739
Epoch 4/8
178/178 [==============================] - 218s 1s/step - loss: 0.2629 - accuracy: 0.8885 - val_loss: 0.2578 - val_accuracy: 0.8902
Epoch 5/8
178/178 [==============================] - 218s 1s/step - loss: 0.2258 - accuracy: 0.9049 - val_loss: 0.2543 - val_accuracy: 0.8884
Epoch 6/8
178/178 [==============================] - 218s 1s/step - loss: 0.1843 - accuracy: 0.9234 - val_loss: 0.2147 - val_accuracy: 0.9105
Epoch 7/8
178/178 [==============================] - 218s 1s/step - loss: 0.1613 - accuracy: 0.9343 - val_loss: 0.2146 - val_accuracy: 0.9116
Epoch 

In [ ]:
model_weight.evaluate(valid_generator_weight,
steps=STEP_SIZE_TEST_WEIGHT)

187/187 [==============================] - 20s 107ms/step - loss: 0.2050 - accuracy: 0.9183


[0.2050064355134964, 0.9182955026626587]

In [ ]:
test_generator_weight.reset()
pred_weight=model_weight.predict_generator(test_generator_weight,
steps=STEP_SIZE_TEST_WEIGHT+1,
verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  after removing the cwd from sys.path.


188/188 [==============================] - 62s 332ms/step


In [ ]:
predicted_class_indices_weight=np.argmax(pred_weight,axis=1)
labels_weight = (train_generator_weight.class_indices)
labels_weight = dict((v,k) for k,v in labels_weight.items())
predictions_weight = [labels_weight[k] for k in predicted_class_indices_weight]

In [ ]:
filenames_weight=test_generator_weight.filenames
results_weight=pd.DataFrame({"id":filenames_weight,
                      "predicted_weight":predictions_weight})
results_weight.to_csv("/content/results_weight.csv",index=False)
model_weight.save('/content/models/model_weight.h5')